In [134]:
import numpy as np
from Crypto.Util.number import getPrime, inverse
from hashlib import sha256
from random import randint
from Crypto.PublicKey import RSA
import math as m

# Generate a keypair for both the sender and receiver

In [135]:
# sender_keypair = RSA.generate(2048) #recommended 2048 key length
# receiver_keypair = RSA.generate(2048)

# pwd = b'secure_password'
# with open("sender_privkey.pem", "wb") as f:
#     data = sender_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [136]:
# pwd = b'secure_password_receiver'
# with open("receiver_privkey.pem", "wb") as f:
#     data = receiver_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [137]:
# with open("receiver_pubkey.pem", "wb") as f:
#     data = receiver_keypair.public_key().export_key()
#     f.write(data)

In [138]:
# with open("sender_pubkey.pem", "wb") as f:
#     data = sender_keypair.public_key().export_key()
#     f.write(data)

# Message (m) and Encryption -> M

In [139]:
msg = 'secret message'.encode('utf-8')
msg

b'secret message'

# Key generation

In [140]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

In [141]:
sender_keypair = generate_keypair()
receiver_keypair = generate_keypair()
print(sender_keypair, '\n', receiver_keypair)

((14238523303469621893728265686321914146255711318456753625759103176072085485467748910995381993346355686383786569934387917597902634439007012103739958587235582809275269018433369957365518749471623860220758371385502497602372991719034824357771880596093653776475399301193626393282541474488839939051438899057748133647090502693539354815108618082295079602504775699287030071961945782724456809652073763576209380505937218523911857496081042861237471366423711409423011868780117398549580720482371262977966255248358855527661150205409458384841459280823293308111082525471154060447847108563533452705321818055269318134403767767932378791361, 2386284585577940724184859107083131551097853861629881886302836106861401398912959254785023796893808279124037571854319403214739173447329627027144422655846477302582514053083040568657018665256536102251683908528644511473451649191199066975277265298802152523021615983402698640039707247457810809243166780265367826588545669349253025834784818882997450197651745123566585398758556185948037833

x -> encoded plaintext
y -> encrypted plaintext
h -> hashed x

# Encoding Message

In [142]:
msg = 'secret message'.encode('utf-8')
x = int.from_bytes(msg, 'big')
x

2340509926146499719798124187838309

In [143]:
hacker_msg = 'not secret'.encode('utf-8')
x_hack = int.from_bytes(hacker_msg, 'big')
x_hack

521516269522829685450100

# Encryption

In [144]:
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n)
y

11048471293126819453676640817821602986368414067173066224353417211733498296966429988228438312779850040959536705160314071691804304497026746850132495197227881202390031401965592905227939687877280438994538595781891511838388665927690181146698786089749892988699379403418771700397151458639864670690867241132408355049774375940265899232937395406775558559416186998452973373032950546001900118927383301221048180758172681484594032286698729449489740561311367055928960207295360502410812406177069558370702841609555775390964115443278549679640659017199727009171733003020974880685164076761292027328579554841064804840327102923652320967719

In [145]:
y_hack = pow(x_hack, e, n)
y_hack

11364098135822158703747025249079381723185141307589582512829037467939661214571455194256327914830986338523772760154733513028740226079241233966276366896461873081233238598266660000227376552983139048946762736430134330433540041787453224814424848057416829203313340727282939261099383037386076990154098388688839307758091861306412034164948553145264866111856875200975069686411927060013223474842091834231879724543744269216650328446243568990136805825582582004379964175967309359966005695835353081488364172432406451812442518353136853671752991764760775873769108405656565669938630393957437200773045048869328976455533166305060140465195

# Hash

In [146]:
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')
h

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Signing

In [147]:
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n)
sign 

12178614395010014029569131115034440343470580302285362273075503085580091711944995097745443935347717858811853080939549367986889015080039334301628930091918676263221463920444232909610774257407051983292313505240279212786835855766440042102222779864387507728386590815669517320002328164325133884922252166714572444065886075024427854551950947736561058297251444887615086603861644783889408831326387980923757715734794474885290922185766202240776626301645829080501153402781936474577740179281651876282033841850694198800473063852152766617435623093112162387526632898159487373821019133455880259073913579204203428682148794158143633454572

# Decryption of hash (sign) with Sender pub key

In [148]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)
decrypt_to_hash

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Decryption of message

In [149]:
n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)
decrypt_to_msg

2340509926146499719798124187838309

# Hash verification 

In [150]:
msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
print(final_h.digest() == hash.digest())

True


# Get plaintext

In [151]:
decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

'secret message'

In [152]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

# Generating keypairs
sender_keypair = generate_keypair() #generate sender keypair
receiver_keypair = generate_keypair() #generate receiver keypair
print(sender_keypair, '\n', receiver_keypair)

#Define message x
msg = 'secret message'.encode('utf-8') #secret message
x = int.from_bytes(msg, 'big') #turn bytes to integer

# Encrypt message x with receiver public key -> ciphertext y
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n) 

# Compute hash
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')

#Signing 
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n) #signing hash with sender private key

((23737634819838997203586896897228371989028885812067357008711294039546698507304745877427609604538273071386426312593523397709690188945143200921327914886686539637127614658520916730147050091996601175834759526712117863781088121755760608597937453884494640217547536561962684913208567130989567634392473963859520894099954281783762980018471971055577493408912517022524938341580335484789910383227512539665941652327293816627098347382128845402291982590686180395298671070655189265663826421681919675646008225794300459152941944262510395809897577814001544094871956113613056780052134106624962912263803488277062429883028268808964127749429, 2233679273432634926085299446645724625031282156238479429584534045802848496454037159343546229016392661303413247905855886062173659977625978352204737396107993681814841346614933393999148729360644277724703245494102460274715946368755663133206617498408337062655920028491421285946163162478840159355112858228102110412272491064836120413979259016750119276297372970939733153942832120746739785

In [153]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)

n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)

msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
final_h_int = int.from_bytes(final_h.digest(), 'big')  

valid_signature = decrypt_to_hash == final_h_int
print("Signature valid?", valid_signature)

decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

Signature valid? True


'secret message'